# Introduction

This notebook will present how to run the code that's in the ScdmsML github repository. https://github.com/LucasFenaux/ScdmsML 

The general structure is the following. The code in the github is divided in multiple experiments, which are meant to train different models on different types of datasets. Each experiment follows roughly the following structure:

*   Pre-Processing
*   Dataloading
*   Model
*   Train, Run and Test code

And I'll close off with a Conclusion

While the Model, Train, Run and Test code is shared most of the time, the Pre-processing and Dataloading is mostly experiment specific.

In this notebook, I will detail and setup the code to run the experiment that can be found in the "run_raw_all_channels.py" file. This notebook can be adapted to run any of the experiments by just substituting the appropriate code where they differ.


For most of our experiments, we used photoneutron data from the Soudan DMC_V1-5 production and therefore this notebook will be tailored towards this particular dataset.

**I would not recommend actually trying to run this notebook locally or using Google Colab as it can be quite demanding both in memory/gpu memory consumption as well as processing power and file storage. This notebook should be used as a tutorial/helper to build/extract the parts of the code you need to either re-run our experiments or run your own new experiments**






# Pre-Processing

The pre-processing can be split up into two parts, that only need to be ran once each to produce the desired files that can then be used for dataloading.

In [ ]:
def pre_processing_part_1():
  """Preprocessing for the raw data files
  Gathers the data from all the different files into a singular matrix, then
  removes all the meta data except the event number and event channel before 
  saving it to a numpy file"""
    # First file is data dump, DO NOT INCLUDE IT
    filepaths = []
    for i in range(2, 977):
        last_part = ""
        if i >= 100:
            last_part += str(i)
        elif i >= 10:
            last_part = last_part + "0" + str(i)
        else:
            last_part = last_part + "00" + str(i)
        last_part += ".gz"
        filepaths.append(
            "path/to/data/folder/libinput_sb-70V_F0" + last_part)   # this was the name of the files in this particular production, could change

    logging.info("getting all events")
    matrix = get_all_events(filepaths)
    logging.info("done getting events")
    logging.info("size of the data matrix {}".format(sys.getsizeof(matrix)))
    # we only care about event number and channel
    logging.info("matrix shape before column deletion {}".format(np.shape(matrix)))
    matrix = np.delete(matrix, [1, 2, 4], axis=1)
    logging.info("matrix shape after deletion {}".format(np.shape(matrix)))
    np.save("path/to/save/folder/pre_processed_data.npy", matrix)

After running the first part, we'll then run the second part that will format the data in the correct way for us to use.
Notice that we have a function to check the format of the given data, if this check fails, I would recommend either writing your own code to get the data in the file pre-processed above in the format wanted by the function or to run the deprecated version of this pre-processing that can be found in the 'run_raw_all_channels.py' file of the github. 

The reason for this format check is that until now, the production files were always in this format and it speeds up the pre-processing process significantly to assume that they are, the deprecated code still works, however, it makes minimal assumptions on the layout of the events in the data matrix and therefore can take quite a long time to run if the amount of data is substantial ($\geq$ 10000 events) 

Also, this pre-processing assumes that we are only using 8 channels, however if it's not the case, it can be easily modified to adapt to any specific number of channels.

In [ ]:
def check_format(data):
    """ Function to check the format of the pre-processed data after the first round of pre-processing
    and verify that it is in the format where we have for each event number, all 8 channel outputs in a row in the same
    order across all events"""
    n = np.shape(data)[0]
    all_event_numbers = data[:, 0]
    all_channel_numbers = data[:, 1]
    channels = []
    first_event = None
    current_event = None
    shape_matches = False
    for i in range(n):
        if first_event is None:
            first_event = all_event_numbers[i]
            current_event = first_event
        if current_event != all_event_numbers[i] and i % len(channels) != 0:
            logging.ERROR("data is not in the correct format, event numbers are not order properly, aborting")
            print("data is not in the correct format, event numbers are not order properly, aborting")
            return False, channels, all_channel_numbers[i], i % 8
        if first_event == current_event:
            channels.append(all_channel_numbers[i])
        elif not shape_matches:
            if i % len(channels) != 0:
                logging.ERROR("data is missing some rows")
                print("data is missing some rows")
                return False, [], 0, 0
            else:
                shape_matches = True
        if channels[i%8] != all_channel_numbers[i]:
            logging.ERROR("data is not in the correct format, channel numbers are not order properly, aborting")
            print("data is not in the correct format, channel numbers are not order properly, aborting")
            return False, channels, all_channel_numbers[i], i % 8

    # if we get here, it means the data is in the correct format
    return True, channels, -1, -1


What this pre-processing part does is that it takes in the event matrix that has the following layout (assumed after using the check format function) for a row:

[event_number|channel_number|timestep 0|timestep 1|....|timestep 4094|timestep 4095]

and the column layout is the following:

[[first event_number|first selected channel|...]

[first event_number|second selected channel]

...

[last even_number| eighth selected channel]]

and turns it into a 3D matrix where each 2D submatrix contains all the 8 channels of each event number together. The final matrix has the following shape: **number of events x length of the sequence (usually 4096) x number of channels (usually 8)**

The code also stores the ordering of the event numbers in a seperate file to figure out the labels during the dataloading part.



In [ ]:
def multi_process_pre_procesing_part_2():
    """ Supposed to do the same thing as the slow pre processing part2, however checks for the format first
    in order to speed up the process. Does require the proper format however
    """
    # First check the format
    print("start loading")
    data = np.load("path/to/save/folder/pre_processed_data.npy")
    print("loaded intial pre-processed data")
    is_valid, channel_order, a, b = check_format(data)
    if not is_valid:
        print(str(channel_order), str(a), str(b))
        return

    def multi_process_pre(submatrix_indices):
        submatrix = data[submatrix_indices[0]:submatrix_indices[1], :]
        all_event_numbers = submatrix[:, 0]
        submatrix = np.delete(submatrix, 0, axis=1)  # remove ev
        submatrix = np.delete(submatrix, 0, axis=1)  # remove channel num
        n = int(np.shape(submatrix)[0]/8) # we shouldn't get any rounding error as the check format checks if we have the correct shape
        submatrix_3D = []
        event_numbers = []
        for i in range(n):
            a = submatrix[i:i+8, :]
            a = np.transpose(a)
            submatrix_3D.append(a)
            event_numbers.append(all_event_numbers[8*i])
        submatrix_3D = np.array(submatrix_3D)
        assert np.shape(submatrix_3D)[0] == len(event_numbers)
        return submatrix_3D, event_numbers

    class Worker(Thread):
        def __init__(self, input_queue, output_queue):
            Thread.__init__(self)
            self.input_queue = input_queue
            self.output_queue = output_queue

        def run(self):
            while True:
                indices = self.input_queue.get()
                try:
                    out_submatrix_3D, out_event_numbers = multi_process_pre(indices)
                    self.output_queue.put((out_submatrix_3D, out_event_numbers))
                finally:
                    self.input_queue.task_done()

    in_queue = Queue()
    out_queue = Queue()
    m = mp.cpu_count()
    for x in range(m):
        worker = Worker(in_queue, out_queue)
        worker.daemon = True
        worker.start()
    rows = np.shape(data)[0]
    chunk_size = int(rows/m)
    while chunk_size%8 != 0:
        chunk_size -= 1  # so that we cut on the event separation line and not before
    for y in range(m):
        # create the indices
        if y == m - 1:
            in_queue.put((y*chunk_size, rows))
        else:
            in_queue.put((y*chunk_size, (y+1)*chunk_size))

    # Now we pick up the finished work and reconstruct the array
    data_3D = None
    events = None
    for z in range(m):
        sub_3D, evs = out_queue.get()
        if data_3D is None and events is None:
            data_3D = sub_3D
            events = evs
        else:
            data_3D = np.concatenate((data_3D, sub_3D), axis=0)
            events.extend(evs)
        out_queue.task_done()
    events = np.array(events)
    np.save("path/to/save/folder/pre_processed_data_3D_all_attribute.npy", data_3D)
    np.save("path/to/save/folder/pre_processed_data_events_all_attributes.npy", events)

Finally, we normalize the resulting matrix 

In [ ]:
def normalizing():
    """ Normalizes the pre-processed data """
    data = np.load("path/to/save/folder/pre_processed_data_3D_all_attribute.npy")
    # this pre-processed data does not contain channel or event number anymore
    normalizer = NDMinMaxScaler()
    normalizer.fit(data)
    print(np.shape(data))
    normalized_data = normalizer.transform(data)
    print(np.shape(normalized_data))

    np.save("path/to/save/folder/pre_processed_normalized_data_3D_all_attribute.npy", normalized_data)


In [ ]:
# This cell shows how the pre-processing should usually be run.
pre_processing_part_1()
multi_process_pre_procesing_part_2()
normalizing()

# Dataloading

Now that we're done with the pre-processing part, we can move on to the dataloading part. If you are unfamiliar with dataloading in Pytorch, I recommend you check out https://pytorch.org/docs/stable/data.html .

The dataloading consists of two main parts. Fetching the labels from the data matrix and then assembling the Dataloader objects.
In order to do this, we use a function called get_num_scatters which given the .mat init file for the experiment returns the scatters, which allows us to construct our ground truth values for the experiment (in our particular case, if it was a single or multiple scatter). This function can be found in the dataloading.py script in our github if you're interested in the specifics or adding it to your own code base.


In [ ]:
# so we first fetch the scatters
def all_channels_raw_data_loader(data_file, event_file, init_path, num_scatter_save_path, det=14):
    all_data = np.load(data_file)
    scatters, single_scatter = get_num_scatters(init_path, save_path=num_scatter_save_path, det=det)
    evs = list(single_scatter.keys())
    targets = []
    all_event_numbers = np.load(event_file)
    target_event_numbers = []
    logging.info("data matrix shape {}".format(np.shape(all_data)))
    assert np.shape(all_data)[1] == 4096

    for i in range(np.shape(all_data)[0]):
        ev = all_event_numbers[i]
        if ev not in evs:
            logging.info("event number {} was not present in the init file and got deleted".format(ev))
            continue
        target_event_numbers.append(ev)
        targets.append(single_scatter[ev])

    if len(np.unique(all_event_numbers)) - len(np.unique(target_event_numbers)) > 0:
        logging.info("{} raw data events were not found in the init file".format(len(all_event_numbers) - len(target_event_numbers)))
    else:
        logging.info("all raw data events were found in the init file")

    return all_data, targets, target_event_numbers

In [ ]:
# and then we assemble the DataLoader objects
def torch_all_channels_raw_data_loader(batch_size=256,num_workers=1, pin_memory=False):
    num_scatter_save_path = os.path.join("../results/files/pca_numscatters.txt")
    data, targets, target_evs = all_channels_raw_data_loader(
        "path/to/save/folder/pre_processed_normalized_data_3D_all_attribute.npy.npy",
        "path/to/save/folder/pre_processed_data_events_all_attributes.npy",
        "path/to/save/folder/PhotoNeutronDMC_InitialTest10K_jswfix.mat",  # the .mat init file
        num_scatter_save_path)
    print(np.min(data), np.max(data))
    train_data, test_data, train_targets, test_targets = train_test_split(data,
                                                                          targets)  # can add target_evs in there if you want to keep track of them as well

    print("train data shape {}".format(np.shape(train_data)))
    print("test data shape {}".format(np.shape(test_data)))
    train_data = torch.Tensor(train_data)
    train_targets = torch.Tensor(train_targets).to(torch.int64)
    train_targets = torch.nn.functional.one_hot(train_targets).to(torch.float)
    # assert torch.max(train_targets) <=1 and torch.min(train_targets) >= 0
    test_data = torch.Tensor(test_data)
    test_targets = torch.Tensor(test_targets).to(torch.int64)
    # assert torch.max(test_targets) <=1 and torch.min(test_targets) >=0
    test_targets = torch.nn.functional.one_hot(test_targets).to(torch.float)

    # logging.info("{}, {}".format(type(train_targets), type(test_targets)))
    train_dataset = TensorDataset(train_data, train_targets)
    train_sampler = RandomSampler(train_dataset)
    train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size, num_workers=num_workers,
                              pin_memory=pin_memory)
    test_dataset = TensorDataset(test_data, test_targets)
    test_sampler = SequentialSampler(test_dataset)
    test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size, num_workers=num_workers,
                             pin_memory=pin_memory)

    return train_loader, test_loader

This sorts out the dataloading. There are a couple of global variable that aren't explicitely set as to not clutter the notebook, if you're wondering how to set them, it is either talked about in the pytorch link a couple cells above or you can check out the details in the github repo. 

# Model

In this section, I will describe a couple of models we used, their strong points and short comings as well as what could be expanded upon.

I will also give a brief explanation of how those model are coded, if you want a more general explanation of how to code models in pytorch, you should checkout  https://pytorch.org/docs/stable/notes/extending.html . It explains their autograd system as well as how each part of the model is coded/works.

In a general sense, when you construct a pytorch model, you need 2 methods. The __init__ and the forward methods. The forward method is what is called when you want to feed data into your model. It usually takes the component initialized in the __init__ method to perform operations on the given data that will then be used during backpropagation automatically with autograd.

## LSTM
So first we have a regular lstm. This works by first passing the input sequence into the LSTMcell, then passing the last hidden state into a feedforward network (here 2 Linear Layers) to produce a prediction.
The upside of this model is that it's simple and works well for short sequences. However in our case, since the sequences in this production are 4096 timesteps long, the LSTM really struggles and performs poorly.


You can also find in the github a Bidirectional LSTM, it works as 3 LSTM cells, 1 processing the sequence forward, another one processing it backwards and a last one processing the output of the first two together and then feeding that into a FeedForward structure (2 Linear Layers here). 


In [ ]:

class LSTMClassifier(nn.Module):
    """
    Version V1.3
    A regular one layer LSTM classifier using LSTMCell -> FFNetwork structure
    """
    def __init__(self, input_dim, hidden_dim, label_size, device=torch.device("cuda"), dropout_rate=0.1):
        super().__init__()
        self.lstm = nn.LSTMCell(input_dim, hidden_dim)
        self.hidden2ff = nn.Linear(hidden_dim,  int(np.sqrt(hidden_dim)))
        self.ff2label = nn.Linear(int(np.sqrt(hidden_dim)), label_size)
        self.hidden_dim = hidden_dim
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_rate)
        self.device = device
        self.initialize_weights()

    def initialize_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)
        for name, param in self.hidden2ff.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)
        for name, param in self.ff2label.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)

    def forward(self, x):

        hs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)
        cs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)

        for i in range(x.size()[1]):
            hs, cs = self.lstm(x[:, i], (hs, cs))

        hs = self.dropout(hs)
        hs = self.hidden2ff(hs)
        return self.sigmoid(self.ff2label(hs)) 

## CNN_LSTM

To improve performance, we then opted to improve our LSTM by adding a Convolutional Net to first perform feature extraction on the input before feeding it into the LSTM. This process has benefits that are two fold. First, it helps reduce the length the sequence from 4096 to the order of 100 timesteps, which is much more manageable for the LSTM. On top of that, it takes the 8 channels (features) we have, filters out noise and useless information for the current task as well as extract important features and information from the sequence. 

For this model, I'm presenting here the latest version of it tested, it does not however mean it is the version that performs the best. The problem we encountered with the first version of the model is that it requires a significantly bigger amount of data to train (we had roughly 40 000 samples, it probably needs at least an order of magnitude more than that) and therefore it was overfitting greatly. To reduce the overfitting, we added extra dropout layers as well as reduced the size of the Convolutional Net to reduce the complexity of the model. However, it still didn't perform well enough **so it would be worth revisiting both versions when aditionnal data has been acquired**


In [ ]:
class CNN_LSTM_Classifier(nn.Module):
    """
    Version V0.3
    CNN+LSTM classifier using ConvNet -> LSTMCell -> FFNetwork structure
    """
    def __init__(self, input_dim, hidden_dim, label_size, device=torch.device("cuda"), dropout_rate=0.1):
        super().__init__()
        self.convnet1 = nn.Sequential(nn.Conv1d(in_channels=input_dim, out_channels=input_dim, kernel_size=7,
                                               padding=2), nn.MaxPool1d(kernel_size=4),
                                     nn.BatchNorm1d(input_dim), nn.ReLU()) # here to reduce noice
        self.convnet2 = nn.Sequential(nn.Conv1d(in_channels=input_dim, out_channels=2*input_dim, kernel_size=5,
                                               padding=2), nn.MaxPool1d(kernel_size=4),
                                     nn.BatchNorm1d(2*input_dim), nn.ReLU())  # start extracting features and reduce size
                                                                              # ro reduce complexity for lstm
        self.convnet3 = nn.Sequential(nn.Conv1d(in_channels=2*input_dim, out_channels=4*input_dim, kernel_size=3,
                                               padding=2), nn.MaxPool1d(kernel_size=2),
                                     nn.BatchNorm1d(4*input_dim), nn.ReLU())
        # removed to reduce model complexity to try and limit overfitting
        # self.convnet4 = nn.Sequential(nn.Conv1d(in_channels=4*input_dim, out_channels=8*input_dim, kernel_size=4,
        #                                        padding=2), nn.BatchNorm1d(8*input_dim), nn.ReLU())
        # self.convnet5 = nn.Conv1d(in_channels=8*input_dim, out_channels=16*input_dim, kernel_size=4, padding=2)
        # self.lstm = nn.LSTMCell(16*input_dim, hidden_dim)
        self.convnet5 = nn.Conv1d(in_channels=4*input_dim, out_channels=4*input_dim, kernel_size=4, padding=2)
        self.lstm = nn.LSTMCell(4*input_dim, hidden_dim)
        self.hiddentoff = nn.Linear(hidden_dim, int(np.sqrt(hidden_dim)))
        self.relu = nn.ReLU()
        self.fftolabel = nn.Linear(int(np.sqrt(hidden_dim)), label_size)
        self.hidden_dim = hidden_dim
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_rate)
        self.device = device
        self.initialize_weights()

    def initialize_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)
        for name, param in self.hiddentoff.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)
        for name, param in self.fftolabel.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)

    def forward(self, x):
        x = torch.reshape(x, (x.size()[0], x.size()[2], x.size()[1]))
        x = self.convnet1(x)
        x = self.convnet2(x)
        x = self.convnet3(x)
        # x = self.convnet4(x)
        x = self.convnet5(x)
        x = torch.reshape(x, (x.size()[0], x.size()[2], x.size()[1]))
        x = self.dropout(x)
        hs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)
        cs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)

        for i in range(x.size()[1]):
            hs, cs = self.lstm(x[:, i], (hs, cs))
            # hs = self.dropout(hs)
            # cs = self.dropout(cs)

        hs = self.dropout(hs)
        hs = self.hiddentoff(hs)
        hs = self.relu(hs)
        # return self.sigmoid(self.hidden2label(hs.reshape(x.shape[0], -1)))
        return self.sigmoid(self.fftolabel(hs))

And here is the original more complex version V0.1 (V0.2 was misslabeled as 0.1 for a commit)

In [ ]:

class CNN_LSTM_Classifier(nn.Module):
    """
    Version V0.1
    CNN+LSTM classifier using ConvNet -> LSTMCell -> FFNetwork structure
    """
    def __init__(self, input_dim, hidden_dim, label_size, device=torch.device("cuda"), dropout_rate=0.1):
        super().__init__()
        self.convnet1 = nn.Sequential(nn.Conv1d(in_channels=input_dim, out_channels=input_dim, kernel_size=4,
                                               padding=2), nn.MaxPool1d(kernel_size=2),
                                     nn.BatchNorm1d(input_dim), nn.ReLU()) # here to reduce noice
        self.convnet2 = nn.Sequential(nn.Conv1d(in_channels=input_dim, out_channels=2*input_dim, kernel_size=4,
                                               padding=2), nn.MaxPool1d(kernel_size=2, stride=2),
                                     nn.BatchNorm1d(2*input_dim), nn.ReLU())  # start extracting features and reduce size
                                                                              # ro reduce complexity for lstm
        self.convnet3 = nn.Sequential(nn.Conv1d(in_channels=2*input_dim, out_channels=4*input_dim, kernel_size=4,
                                               padding=2), nn.MaxPool1d(kernel_size=2),
                                     nn.BatchNorm1d(4*input_dim), nn.ReLU())
        self.convnet4 = nn.Sequential(nn.Conv1d(in_channels=4*input_dim, out_channels=8*input_dim, kernel_size=4,
                                               padding=2), nn.BatchNorm1d(8*input_dim), nn.ReLU())
        self.convnet5 = nn.Conv1d(in_channels=8*input_dim, out_channels=16*input_dim, kernel_size=4, padding=2)
        self.lstm = nn.LSTMCell(16*input_dim, hidden_dim)
        self.hiddentoff = nn.Linear(hidden_dim, int(np.sqrt(hidden_dim)))
        self.relu = nn.ReLU()
        self.fftolabel = nn.Linear(int(np.sqrt(hidden_dim)), label_size)
        self.hidden_dim = hidden_dim
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_rate)
        self.device = device
        self.initialize_weights()

    def initialize_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)
        for name, param in self.hiddentoff.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)
        for name, param in self.fftolabel.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0001)
            elif 'weight' in name:
                nn.init.xavier_normal_(param)

    def forward(self, x):
        x = torch.reshape(x, (x.size()[0], x.size()[2], x.size()[1]))
        x = self.convnet1(x)
        x = self.convnet2(x)
        x = self.convnet3(x)
        x = self.convnet4(x)
        x = self.convnet5(x)
        x = torch.reshape(x, (x.size()[0], x.size()[2], x.size()[1]))
        x = self.dropout(x)
        hs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)
        cs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)

        for i in range(x.size()[1]):
            hs, cs = self.lstm(x[:, i], (hs, cs))
            # hs = self.dropout(hs)
            # cs = self.dropout(cs)

        hs = self.dropout(hs)
        hs = self.relu(hs)
        hs = self.hiddentoff(hs)

        # return self.sigmoid(self.hidden2label(hs.reshape(x.shape[0], -1)))
        return self.sigmoid(self.fftolabel(hs))

To summarize, those models performed much better on the train set (reaching 90% accuracy for V0.3 and 98% accuracy for V0.1) however had a very hard time generalizing their learning (only ~55% accuracy for both) on the validation set.

## CNN_LSTM with Attention

This is an even more complex version of the CNN_LSTM model that **hasn't been ran or tested yet**. This is here only to give inspiration of what could be done to improve the model's performance even further once more data is available. This model is a CNN_LSTM that uses a part of Transformer models called Attention coupled with a Encoder/Decoder structure, which allows the model to focus and extract more information on important parts of the sequence. This one uses the V0.3 structure of the CNN_LSTM.

The upsides are that it is even more powerful than the CNN_LSTM, however, since it has at least twice the amount of parameters, it will need even more data to train properly so it might have difficulties generalizing.

To learn more about Attention, how it works and how it can be used, I recommend reading https://arxiv.org/abs/1706.03762 .

In [ ]:

class AdditiveAttention(nn.Module):
    def __init__(self, hidden_size):
        super(AdditiveAttention, self).__init__()

        self.hidden_size = hidden_size

        # A two layer fully-connected network
        # hidden_size*2 --> hidden_size, ReLU, hidden_size --> 1
        self.attention_network = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

        self.softmax = nn.Softmax(dim=1)

    def forward(self, queries, keys, values):
        """The forward pass of the additive attention mechanism.

        Arguments:
            queries: The current decoder hidden state. (batch_size x hidden_size)
            keys: The encoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)
            values: The encoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)

        Returns:
            context: weighted average of the values (batch_size x 1 x hidden_size)
            attention_weights: Normalized attention weights for each encoder hidden state. (batch_size x seq_len x 1)

            The attention_weights must be a softmax weighting over the seq_len annotations.
        """
        batch_size = keys.size(0)
        expanded_queries = queries.view(batch_size, -1, self.hidden_size).expand_as(keys)
        concat_inputs = torch.cat([expanded_queries, keys], dim=2)
        unnormalized_attention = self.attention_network(concat_inputs)
        attention_weights = self.softmax(unnormalized_attention)
        context = torch.bmm(attention_weights.transpose(2, 1), values)
        return context, attention_weights


class CNN_LSTM_Attention_Classifier(nn.Module):
    """
    Version V0.0
    CNN+LSTM classifier using ConvNet -> LSTMCell + Attention -> FFNetwork structure
    """
    class ConvNet(nn.Module):
        def __init__(self, input_dim, dropout_rate):
            super().__init__()
            self.convnet1 = nn.Sequential(nn.Conv1d(in_channels=input_dim, out_channels=input_dim, kernel_size=7,
                                                    padding=2), nn.MaxPool1d(kernel_size=4),
                                          nn.BatchNorm1d(input_dim), nn.ReLU())  # here to reduce noice
            self.convnet2 = nn.Sequential(nn.Conv1d(in_channels=input_dim, out_channels=2 * input_dim, kernel_size=5,
                                                    padding=2), nn.MaxPool1d(kernel_size=4),
                                          nn.BatchNorm1d(2 * input_dim),
                                          nn.ReLU())  # start extracting features and reduce size
            # to reduce complexity for lstm
            self.convnet3 = nn.Sequential(
                nn.Conv1d(in_channels=2 * input_dim, out_channels=4 * input_dim, kernel_size=3,
                          padding=2), nn.MaxPool1d(kernel_size=2),
                nn.BatchNorm1d(4 * input_dim), nn.ReLU())
            # removed to reduce model complexity to try and limit overfitting
            # self.convnet4 = nn.Sequential(nn.Conv1d(in_channels=4*input_dim, out_channels=8*input_dim, kernel_size=4,
            #                                        padding=2), nn.BatchNorm1d(8*input_dim), nn.ReLU())
            # self.convnet5 = nn.Conv1d(in_channels=8*input_dim, out_channels=16*input_dim, kernel_size=4, padding=2)
            # self.lstm = nn.LSTMCell(16*input_dim, hidden_dim)
            self.convnet5 = nn.Conv1d(in_channels=4 * input_dim, out_channels=4 * input_dim, kernel_size=4, padding=2)
            self.dropout = nn.Dropout(dropout_rate)

        def forward(self, x):
            x = torch.reshape(x, (x.size()[0], x.size()[2], x.size()[1]))
            x = self.convnet1(x)
            x = self.convnet2(x)
            x = self.convnet3(x)
            # x = self.convnet4(x)
            x = self.convnet5(x)
            x = torch.reshape(x, (x.size()[0], x.size()[2], x.size()[1]))
            x = self.dropout(x)
            return x

    class CNN_LSTM_Encoder(nn.Module):
        def __init__(self, input_dim, hidden_dim, dropout_rate):
            super().__init__()
            self.convnet = self.ConvNet(input_dim, dropout_rate)
            self.lstm = nn.LSTMCell(4*input_dim, hidden_dim)

        def forward(self, x):
            hs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)
            cs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)

            x = self.convnet(x)
            annotations = []

            for i in range(x.size()[1]):
                hs, cs = self.lstm(x[:, i], (hs, cs))
                annotations.append(hs)

            annotations = torch.stack(annotations, dim=1)
            return annotations, hs, cs

        def initialize_weights(self):
            for name, param in self.lstm.named_parameters():
                if 'bias' in name:
                    nn.init.constant_(param, 0.0001)
                elif 'weight' in name:
                    nn.init.xavier_normal_(param)

    class CNN_LSTM_Decoder(nn.Module):
        def __init__(self, input_dim, hidden_dim, label_size, dropout_rate=0.1):
            super().__init__()
            self.convnet = self.ConvNet(input_dim, dropout_rate)
            self.lstm = nn.LSTMCell(8 * input_dim, hidden_dim)
            self.hiddentoff = nn.Linear(hidden_dim, int(np.sqrt(hidden_dim)))
            self.relu = nn.ReLU()
            self.fftolabel = nn.Linear(int(np.sqrt(hidden_dim)), label_size)
            self.hidden_dim = hidden_dim
            self.sigmoid = nn.Sigmoid()
            self.dropout = nn.Dropout(dropout_rate)
            self.attention = AdditiveAttention(hidden_size=hidden_dim)
            self.initialize_weights()

        def initialize_weights(self):
            for name, param in self.lstm.named_parameters():
                if 'bias' in name:
                    nn.init.constant_(param, 0.0001)
                elif 'weight' in name:
                    nn.init.xavier_normal_(param)
            for name, param in self.hiddentoff.named_parameters():
                if 'bias' in name:
                    nn.init.constant_(param, 0.0001)
                elif 'weight' in name:
                    nn.init.xavier_normal_(param)
            for name, param in self.fftolabel.named_parameters():
                if 'bias' in name:
                    nn.init.constant_(param, 0.0001)
                elif 'weight' in name:
                    nn.init.xavier_normal_(param)

        def forward(self, x, annotations, h_init, c_init):
            x = self.convnet(x)
            # hs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)
            # cs = torch.zeros(x.size(0), self.hidden_dim).to(self.device)
            hs = h_init
            cs = c_init

            for i in range(x.size()[1]):
                context, attention_weights = self.attention(hs, annotations, annotations)
                cur_input = x[:, i]
                cur_input = torch.cat([cur_input, context.squeeze(1)], dim=1)
                hs, cs = self.lstm(cur_input, (hs, cs))

            hs = self.dropout(hs)
            hs = self.hiddentoff(hs)
            hs = self.relu(hs)
            # return self.sigmoid(self.hidden2label(hs.reshape(x.shape[0], -1)))
            return self.sigmoid(self.fftolabel(hs))

    def __init__(self, input_dim, hidden_dim, label_size, device=torch.device("cuda"), dropout_rate=0.1):
        super().__init__()
        self.device = device
        self.encoder = self.CNN_LSTM_Encoder(input_dim, hidden_dim, dropout_rate)
        self.decoder = self.CNN_LSTM_Decoder(input_dim, hidden_dim, label_size,dropout_rate)

    def forward(self, x):
        encoder_annotations, encoder_hidden, encoder_cell = self.encoder(x)
        decoder_outputs = self.decoder(x, encoder_annotations, encoder_hidden, encoder_cell)
        return decoder_outputs

## Transformer

Finally, the last model that could be worth testing once more data is available is a transformer itself. I don't have an implementation of it ready, however, as time passes and since it's a fairly new and popular model, it will be easy to find existing implementations online and adapt them to the particular task at hand. Highly recommend looking into as soon as possible. 

Upsides are that it allows for better parallelization than the LSTM structure and therefore would train much faster on a larger dataset, however it requires more memory to train. It also suffers from being a complex model to train and requiring a large amount of data to train properly.

A good first step to learn more about transformers would be to check out https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html .

# Train, Run and Test Code

We now come to the code that pieces it all together. The train code is the code that will implement what we like to call the "training loop" to train the model.
This training loop consists of fetching the batches of data we want to train our model on, feeding them to our model and using the predicted output with the ground truth labels to compute a loss. Then we backpropagate the gradient of the loss w.r.t the different parts of the model through the model using whatever loss/optimizer prefered.

The run code is the code that pieces all the different elements together: initializes the model, loads the datasets, sets up the iteration procedure for the train loop etc...

And finally the test code is what allows us to validate our model and observe how it performs on new data it hasn't seen before.

## Train Code

For the train code, there are multiple ways to implement it. I will cover the two that I know off. We have:

*   Coding it manually
*   Torch Ignite

The loop can be and usually is coded manually. This is good to do when you want to add custom behavior to your training that can deviate from the norm. However, if the training process is straightforward and is virtually the same as the basic training function that I will describe below, then it can be good to instead use torch ignite instead.




In [ ]:

def train_nn(batch_loader: DataLoader, model: torch.nn.Module, criterion, optimizer, testing: bool,
             device: torch.device):
  '''A standard training loop'''
    if testing:
        model.eval()
        #bar = Bar('Testing', max=len(batch_loader))
    else:
        model.train()
        #bar = Bar('Training', max=len(batch_loader))

    # Progress bar stuff
    #batch_time = AverageMeter()
    #data_time = AverageMeter()
    losses = AverageMeter()    # can be found in the utils/misc.py code file
    #end = time.time()
    for batch_idx, (inputs, target) in enumerate(batch_loader):
        # Measure data loading time
        #data_time.update(time.time() - end)
        inputs = inputs.to(device)
        target = target.to(device)
        output = model(inputs)

        total_loss = criterion(output, target)

        # Record loss
        losses.update(total_loss.item(), inputs.size(0))

        if not testing:
            # Compute gradient and do SGD step
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

        # Measure elapsed time
        #batch_time.update(time.time() - end)
        #end = time.time()

        # plot progress
        #bar.suffix = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | Loss: {loss:.4f} | Gen Loss: {gen_loss: .4f} | Action Loss: {action_loss: .4f}'.format(
            #batch=batch_idx + 1,
            #size=len(batch_loader),
            #data=data_time.avg,
            #bt=batch_time.avg,
            #total=bar.elapsed_td,
            #loss=losses.avg,
            #gen_loss=float(0),
            #action_loss=float(total_loss)
        #)
        #bar.next()

    #bar.finish()
    return losses.avg

### Torch Ignite

To learn the details of how torch ignite works, I recommend you check out https://pytorch.org/ignite/index.html for more details. 

What ignite does is that it helps simplify and streamline the training process of networks and allows for easy re-use of code across the board.
It also has a nice integration with tensorboard (https://pytorch.org/docs/stable/tensorboard.html) which allows for an easier result logging process.

All the torch ignite code is setup in the run file (file we actually run) so I will first detail how to run a regular network without torch ignite and tensorboard.

## Run Code/Test code
So now that we have the train code, we can talk about the actual run file itself and how it is setup.
First you obviously want to import all the code that we discussed above so it can be used to train your model.
I will then detail the steps and code in the code box below
The test code is contained within the run code and the train code, hence it doesn't require an extra section. However, when you're not using ignite, it does require the creation of a "test function", i.e a function that will compute the particular metric you want to look into. We have a basic accuracy computation function defined below.


In [ ]:
def compute_accuracy(predictions, targets):
    """Computes an accuracy based on the given predictions and targets"""
    assert len(predictions) == len(targets)

    accuracy_array = []

    if len(np.shape(targets)) == 1:

        for (idx, pred), (_, t) in zip(enumerate(predictions), enumerate(targets)):
            accuracy_array.append(1) if pred == t else accuracy_array.append(0)

    else:

        for (idx, pred), (_, t) in zip(enumerate(predictions), enumerate(targets)):
            accuracy_array.append(1) if np.argmax(pred) == np.argmax(t) else accuracy_array.append(0)

    accuracy = float(sum(accuracy_array))/float(len(accuracy_array))

    return accuracy

In [ ]:
def run():
    # we do this to make sure gpu training is available, it is highly recommended to train the models on gpu 
    # assert torch.cuda.is_available()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    # Dataloading parameters
    pin_memory = (device.type == "cuda")
    num_workers = 8   # this represents the number of cpu threads you want to assign to the dataloading, the more the better
    batch_size = 2048   # the size of the batches that are fed to the model

    # Model input parameters
    input_size = 8    # the number of channels in the input data
    hidden_size = 50    # how many neurons we want your hidden layers to have, more means a more complex model
    dropout_rate = 0.3    # the probability that any given input will be replaced by a 0 at the dropout layers of the model (should usually be between 0.1 and 0.5 depending on the complexity of the model)

    # Training parameters
    epochs = 1000     # the number of training loops
    learning_rate = 0.001   # the learning rate, usually initialized between 0.01 and 0.0001

    # we initialize the model
    nn = CNN_LSTM_Classifier(input_size, hidden_size, label_size=2, device=device, dropout_rate=dropout_rate)
    nn = nn.to(device) # put it on the gpu if a gpu is available

    # we get the dataloaders 
    train_loader, test_loader = torch_all_channels_raw_data_loader(batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)

    # we initialize the optimizer that will train our model, here we use Adam
    # but there is usually a prefered optimizer for each model, this can be easily found on Google.
    # By default, Adam is a good all-round optimizer that can work for most models
    optimizer = optim.Adam(nn.parameters(), lr=learning_rate, weight_decay=0.0001)

    # the loss function, will depend on the data you are working with,
    # the number of classes and their type as well as personal preference (again here, Google is your friend)
    criterion = torch.nn.BCELoss()

    # Now we arrive to the training/testing code

    # We'll define an extra variable, that is there to set how often you want to validate your model during training
    validation_rate = 5   # we'll set it to every 5 epochs for now\
    validation_counter = 1
    # We iterate over the decided number of epochs
    for i in range(epochs):
        # we first train the model
        train_loss = train_nn(train_loader, nn, criterion, optimizer, testing=False, device=device)
        print("Epoch {} | Training Loss {}".format(i, train_loss))

        # we check if we need to validate the model
        if validation_counter == validation_rate:
            # we don't calculate gradients since we are testing the model
            with torch.no_grad():
                test_loss = train_nn(test_loader, nn, criterion, optimizer, testing=True, device=device)
                print("Epoch {} | Test Loss {}".format(i, test_loss))
            validation_counter = 1
        else:
            validation_counter += 1

    # Now that we are done training the model, we can actually test it using whichever testing function you decided to define
    accuracy = AverageMeter()
    for batch_idx, (inputs, target) in enumerate(test_loader):
        inputs = inputs.to(device)
        target = target.to(device)
        output = model(inputs)

        acc = compute_accuracy(output.numpy(), target.numpy())
        accuracy.update(acc, inputs.size(0))

  print('final test accuracy: {}'.format(accuracy.avg))

### Torch Ignite

Now I'll present the same run/train/test code using pytorch ignite with tensorboard.
First for conveniance, I'm going to share a function that allows for easy creation of tensorboard log directories. 

In [ ]:
def get_tensorboard_log_dir():
    if not path.exists('../results/files/tb_logs'):
        return path.join("../results/files/tb_logs")
    i = 1
    while path.exists('../results/files/tb_logs_{}'.format(i)):
        i += 1
    return path.join("../results/files/tb_logs_{}".format(i))

We'll now setup the event handler for ignite. It catches whatever trigger we want to feed it and then performs custom actions on them. For example, the first trigger below is triggered whenever an epoch is completed and the log_training_loss function is then called. This function prints the loss to standard out and logs it to the current tensorboard summary writer.

In [ ]:
def setup_event_handler(trainer, evaluator, train_loader, test_loader):
    log_interval = 10

    writer = SummaryWriter(log_dir=log_dir)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_loss(trainer):
        print("Epoch[{}] Loss: {:.5f}".format(trainer.state.epoch, trainer.state.output))
        writer.add_scalar("training_iteration_loss", trainer.state.output, trainer.state.epoch)

    @trainer.on(Events.EPOCH_COMPLETED(every=log_interval))
    def log_training_results(trainer):
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        print("Training Results - Epoch: {}  Accuracy: {:.5f} Loss: {:.5f}"
                     .format(trainer.state.epoch, metrics["accuracy"], metrics["nll"]))
        writer.add_scalar("training_loss", metrics["nll"], trainer.state.epoch)
        writer.add_scalar("training_accuracy", metrics["accuracy"], trainer.state.epoch)

    @trainer.on(Events.EPOCH_COMPLETED(every=log_interval))
    def log_testing_results(trainer):
        evaluator.run(test_loader)
        metrics = evaluator.state.metrics
        print("Validation Results - Epoch: {}  Accuracy: {:.5f} Loss: {:.5f}"
                     .format(trainer.state.epoch, metrics["accuracy"], metrics["nll"]))
        writer.add_scalar("testing_loss", metrics["nll"], trainer.state.epoch)
        writer.add_scalar("testing_accuracy", metrics["accuracy"], trainer.state.epoch)


We could see above the presence of both a trainer and evaluator object. Those objects are the base of the ignite system. They take in your model, optimizer and criterion (as well as a metric function for the evaluator) and then take care of all the training/evaluation code.

Below, we have the rest of the run code, as you can see, the hyperparameter initialization remains pretty much the same, however the second part is much shorter and we don't need to write out own train function anymore.

In [ ]:
def run():
    # we do this to make sure gpu training is available, it is highly recommended to train the models on gpu 
    # assert torch.cuda.is_available()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    # Dataloading parameters
    pin_memory = (device.type == "cuda")
    num_workers = 8   # this represents the number of cpu threads you want to assign to the dataloading, the more the better
    batch_size = 2048   # the size of the batches that are fed to the model

    # Model input parameters
    input_size = 8    # the number of channels in the input data
    hidden_size = 50    # how many neurons we want your hidden layers to have, more means a more complex model
    dropout_rate = 0.3    # the probability that any given input will be replaced by a 0 at the dropout layers of the model (should usually be between 0.1 and 0.5 depending on the complexity of the model)

    # Training parameters
    epochs = 1000     # the number of training loops
    learning_rate = 0.001   # the learning rate, usually initialized between 0.01 and 0.0001

    # we initialize the model
    nn = CNN_LSTM_Classifier(input_size, hidden_size, label_size=2, device=device, dropout_rate=dropout_rate)
    nn = nn.to(device) # put it on the gpu if a gpu is available

    # we get the dataloaders 
    train_loader, test_loader = torch_all_channels_raw_data_loader(batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)

    # we initialize the optimizer that will train our model, here we use Adam
    # but there is usually a prefered optimizer for each model, this can be easily found on Google.
    # By default, Adam is a good all-round optimizer that can work for most models
    optimizer = optim.Adam(nn.parameters(), lr=learning_rate, weight_decay=0.0001)

    # the loss function, will depend on the data you are working with,
    # the number of classes and their type as well as personal preference (again here, Google is your friend)
    criterion = torch.nn.BCELoss()

    # we now initialize the trainer
    trainer = create_supervised_trainer(nn, optimizer, criterion, device=device)

    # I have to define an extra function here that will take in the output of my model and format it in a proper format
    # for us to feed it into the Accuracy metric function
    def ot_func(output):
        y_pred, y = output
        y_pred = torch.nn.functional.one_hot(torch.max(y_pred, 1)[1], num_classes=2).to(torch.float)
        return (y_pred, y)

    # here we define the dictionnary of all the metrics we want our evaluator to look at,
    # for now I just used the Accuracy function that can be found in ignite.metrics as well as the loss
    # many more metrics can be found in the ignite.metrics package
    val_metrics = {
        "accuracy": Accuracy(output_transform=partial(ot_func)),
        "nll": Loss(criterion)
    }

    # now that we have all the pre-requisits set up, we can initialize the evaluator
    evaluator = create_supervised_evaluator(nn, metrics=val_metrics, device=device)

    # we now call the setup for the triggers defined previously
    setup_event_handler(trainer, evaluator, train_loader, test_loader)

    # and finally we can just train the model for the specified number of epochs by just calling the run function
    trainer.run(train_loader, max_epochs=epochs)

# Conclusion

To conclude this notebook, I first to note that if you are confused about any part of this code, then feel free to get in contact with me (you can find my most recent contact info on https://www.fenaux.ca/).
Secondly, one part I didn't really cover was the imports, as I didn't want to make the notebook too convoluted. However, if you want to see how it's done, you can check out the original code files on the github repository.